Titanic Machine Learning Data Trainer
- In this notebook, we will analyze the test data and attempt to predict the outcomes for the missing people. Given the data we have, can we accurately predict whether the rest of the people lived or died?
- First, we will want to do manual research on the data we have. Are there any trends (gender, ticket class, age, etc.) that represent a strong-ish correlation with survival rate?

Data Information

(1) train.csv
- train.csv contains the details of a subset of the passengers on board (891 passengers, to be exact -- where each passenger gets a different row in the table).

- The values in the second column ("Survived") can be used to determine whether each passenger survived or not:
- if it's a "1", the passenger survived.
- if it's a "0", the passenger died.

(2) test.csv
- Using the patterns you find in train.csv, you have to predict whether the other 418 passengers on board (in test.csv) survived.

- Note that test.csv does not have a "Survived" column - this information is hidden from you, and how well you do at predicting these hidden values will determine how highly you score in the competition!

(3) gender_submission.csv
- The gender_submission.csv file is provided as an example that shows how you should structure your predictions. It predicts that all female passengers survived, and all male passengers died. Your hypotheses regarding survival will probably be different, which will lead to a different submission file. But, just like this file, your submission should have:

- a "PassengerId" column containing the IDs of each passenger from test.csv.
- a "Survived" column (that you will create!) with a "1" for the rows where you think the passenger survived, and a "0" where you predict that the passenger died.



In [155]:
import numpy as np
import pandas as pd
import os
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [96]:
test_data = pd.read_csv("test.csv")
test_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [83]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [84]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)
print("% of women who survived:", rate_women)

% of men who survived: 0.18890814558058924
% of women who survived: 0.7420382165605095


Given the above rates when comparing men and women, it is clear that gender is a highly correlated attribute to survival rate. Let's dive a little deeper into this, and see if age has any significant correlation.

In [85]:
child = train_data.loc[train_data.Age < 18]["Survived"]
rate_child = sum(child)/len(child)

print("% of children who survived:", rate_child)

% of children who survived: 0.5398230088495575


With a survival rate of ~50% for children (both male and female) under the age of 18, this does not tell us much. However, I have a feeling that this rate could be skewed by males that were closer to that age threshold (i.e. teenage boys). I could see a world where boys that were old enough to not be considered "children" may have not had priority spots on the lifeboats. Let's dig into this a bit further, and see if we can uncover any trends when considering age and gender:

In [89]:
child_male_sub_5 = train_data.loc[train_data.Age < 5][train_data.Sex == 'male']["Survived"]
rate_child_male_sub_5 = sum(child_male_sub_5)/len(child_male_sub_5)

child_male_sub_12 = train_data.loc[train_data.Age < 12][train_data.Sex == 'male']["Survived"]
rate_child_male_sub_12 = sum(child_male_sub_12)/len(child_male_sub_12)

child_male_sub_18 = train_data.loc[train_data.Age < 18][train_data.Sex == 'male']["Survived"]
rate_child_male_sub_18 = sum(child_male_sub_18)/len(child_male_sub_18)

print("% of boys who survived (under 5 years old):", rate_child_male_sub_5)
print("% of boys who survived (under 12 years old):", rate_child_male_sub_12)
print("% of boys who survived (under 18 years old):", rate_child_male_sub_18)

% of boys who survived (under 5 years old): 0.6521739130434783
% of boys who survived (under 11 years old): 0.5555555555555556
% of boys who survived (under 18 years old): 0.39655172413793105


/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_51916/2443884224.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  child_male_sub_5 = train_data.loc[train_data.Age < 5][train_data.Sex == 'male']["Survived"]
/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_51916/2443884224.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  child_male_sub_12 = train_data.loc[train_data.Age < 12][train_data.Sex == 'male']["Survived"]
/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_51916/2443884224.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  child_male_sub_18 = train_data.loc[train_data.Age < 18][train_data.Sex == 'male']["Survived"]


Given these survival rates, it is evident that younger boys that could be considered babies/infants had a much higher chance of survival than older boys. Let's take a look at these same rates for girls:

In [95]:
child_female_sub_5 = train_data.loc[train_data.Age < 5][train_data.Sex == 'female']["Survived"]
rate_child_female_sub_5 = sum(child_female_sub_5)/len(child_female_sub_5)

child_female_sub_12 = train_data.loc[train_data.Age < 12][train_data.Sex == 'female']["Survived"]
rate_child_female_sub_12 = sum(child_female_sub_12)/len(child_female_sub_12)

child_female_sub_18 = train_data.loc[train_data.Age < 18][train_data.Sex == 'female']["Survived"]
rate_child_female_sub_18 = sum(child_female_sub_18)/len(child_female_sub_18)

print("% of girls who survived (under 5 years old):", rate_child_female_sub_5)
print("% of girls who survived (under 12 years old):", rate_child_female_sub_12)
print("% of girls who survived (under 18 years old):", rate_child_female_sub_18)

% of girls who survived (under 5 years old): 0.7058823529411765
% of girls who survived (under 11 years old): 0.59375
% of girls who survived (under 18 years old): 0.6909090909090909


/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_51916/1956758124.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  child_female_sub_5 = train_data.loc[train_data.Age < 5][train_data.Sex == 'female']["Survived"]
/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_51916/1956758124.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  child_female_sub_12 = train_data.loc[train_data.Age < 12][train_data.Sex == 'female']["Survived"]
/var/folders/qr/nj1x3v_n3g1c86l2ynq5dqkc0000gp/T/ipykernel_51916/1956758124.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  child_female_sub_18 = train_data.loc[train_data.Age < 18][train_data.Sex == 'female']["Survived"]


Although there is a strange dip in the survival rate of girls between ages 5-11, it is notable that the survival rate of female babies/infants is very similar to the overall rate of girls under the age of 18. This goes to show that the age discrepancy between female childeren did not share the same impact on survival rate that it did on male children.

Overall, it is clear that the age of the passenger has significant correlation with the survival rate, especially when looking at gender and age together. We will want to keep that in mind when training our data. 

In [88]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('results-random-forest.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


After submitting this results CSV file to Kaggle, we discovered that our Random Forest Classifier model successfully predicted the survival of 77.5% of our test data. While this is reasonably accurate, ideally we'd like to increase that accuracy rate by a bit. Let's try creating a sequential model using Keras and see how the results compare.

In [134]:
keras_train_data = pd.read_csv("train.csv")
keras_test_data = pd.read_csv("test.csv")

# drop the fields we don't care about from the data
drop_fields = ['Ticket', 'Cabin', 'Name']
def clean_data(data, droppable):
    if pd.Series(droppable).isin(data.columns).all():  
        for field in droppable:
            data.drop(field, axis=1, inplace=True)
    return data

clean_data(keras_train_data, drop_fields)
clean_data(keras_test_data, drop_fields)

keras_train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


Now that we've cleaned our data, let's convert the categorical variables to numeric values. Our categorical variables are Sex and Embarked (both nominal). We'll use the get_dummies() function for this.

In [152]:
def convert_features(data, features):
    if pd.Series(features).isin(data.columns).all():
        data = pd.get_dummies(data, columns=features, drop_first=True)
    return data

nominal_features = ['Sex', 'Embarked']
keras_train_data = convert_features(keras_train_data, nominal_features)
keras_test_data = convert_features(keras_test_data, nominal_features)

# fill null values if any exist
keras_train_data.fillna(keras_train_data.mean(), inplace=True)
keras_test_data.fillna(keras_test_data.mean(), inplace=True)

# make sure the columns are aligned
keras_test_data = keras_test_data[keras_train_data.columns.drop('Survived')]

keras_test_data = keras_test_data.astype(np.float32)
keras_train_data = keras_train_data.astype(np.float32)

print(f'keras_train_data shape: {keras_train_data.shape}')
print(f'keras_test_data shape: {keras_test_data.shape}')

keras_train_data.head()


keras_train_data shape: (891, 10)
keras_test_data shape: (418, 9)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1.0,0.0,3.0,22.0,1.0,0.0,7.250000,1.0,0.0,1.0
1,2.0,1.0,1.0,38.0,1.0,0.0,71.283302,0.0,0.0,0.0
2,3.0,1.0,3.0,26.0,0.0,0.0,7.925000,0.0,0.0,1.0
3,4.0,1.0,1.0,35.0,1.0,0.0,53.099998,0.0,0.0,1.0
4,5.0,0.0,3.0,35.0,0.0,0.0,8.050000,1.0,0.0,1.0


In [158]:
X_train = keras_train_data.drop('Survived', axis=1)
y_train = keras_train_data['Survived']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(keras_test_data)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_train, test_size=0.2, random_state=222)

model = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=5, epochs=25, validation_split=0.2)

predictions = (model.predict(X_test_scaled) > 0.5).astype(int).flatten()

submission = pd.DataFrame({'PassengerId': test_data['PassengerId'].astype('int32'), 'Survived': predictions})
submission.to_csv('results-keras.csv', index=False)

Epoch 1/25
114/114 [==============================] - 2s 4ms/step - loss: 0.6777 - accuracy: 0.5835 - val_loss: 0.5627 - val_accuracy: 0.7762
Epoch 2/25
114/114 [==============================] - 0s 2ms/step - loss: 0.5775 - accuracy: 0.7153 - val_loss: 0.4496 - val_accuracy: 0.8322
Epoch 3/25
114/114 [==============================] - 0s 2ms/step - loss: 0.5811 - accuracy: 0.7258 - val_loss: 0.4496 - val_accuracy: 0.8182
Epoch 4/25
114/114 [==============================] - 0s 3ms/step - loss: 0.4975 - accuracy: 0.7733 - val_loss: 0.4254 - val_accuracy: 0.8252
Epoch 5/25
114/114 [==============================] - 0s 2ms/step - loss: 0.5208 - accuracy: 0.7627 - val_loss: 0.4293 - val_accuracy: 0.8042
Epoch 6/25
114/114 [==============================] - 0s 2ms/step - loss: 0.4842 - accuracy: 0.7909 - val_loss: 0.4123 - val_accuracy: 0.8322
Epoch 7/25
114/114 [==============================] - 0s 2ms/step - loss: 0.4766 - accuracy: 0.7856 - val_loss: 0.4159 - val_accuracy: 0.8392
Epoch 